In [33]:
import pandas as pd
import json
import numpy as np

with open('data.json', 'r', encoding='utf-8') as json_data:
    data = json.load(json_data)

normalized_data = []

for key, value in data.items():
    user_id = key
    info = value.get('info', [{}])[0]  
    groups = value.get('groups', [])
    if not groups:
        record = {**info, 'id': user_id}
        normalized_data.append(record)
        continue
    for group in groups:
        record = {**info, **group, 'id': user_id, 'group_id': group['id']}
        normalized_data.append(record)

df = pd.DataFrame(normalized_data, index=False)

In [34]:
df.head()

,id,first_name,last_name,can_access_closed,is_closed,deactivated,name,screen_name,type,photo_50,photo_100,photo_200,group_id
0,114668685,Alexey,Dozorov,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,True,False,deleted,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118465737,Renat,Khakimov,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df = df.drop(['photo_50', 'photo_100', 'photo_200', 'deactivated', 'can_access_closed', 'is_closed', 'type'], axis=1)

In [36]:
df

,id,first_name,last_name,name,screen_name,group_id
0,114668685,Alexey,Dozorov,NaN,NaN,NaN
1,64172662,Alexander,Antonnikov,NaN,NaN,NaN
2,256740,Mikhail,Podgorny,NaN,NaN,NaN
3,118465737,Renat,Khakimov,NaN,NaN,NaN
4,90084913,Anna,Solovskaya,NaN,NaN,NaN
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304.0
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318.0
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575.0
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234.0


In [37]:
df['group_id'] = df['group_id'].astype(float).astype('Int64')

In [38]:
df.rename(columns={'name':'group_name', 'screen_name': 'group_screen_name', 'id': 'user_id', 'first_name': 'user_first_name', 'last_name': 'user_last_name'}, inplace=True)

In [39]:
df.dropna(how='any', inplace=True)

In [40]:
df

,user_id,user_first_name,user_last_name,group_name,group_screen_name,group_id
7,125402825,Artyom,Amosov,Sperry Ukraine,sperrytopsiderukraine,19603458
8,125402825,Artyom,Amosov,Dubstep set up,dubstepforum,22778924
9,125402825,Artyom,Amosov,КИНОМАНИЯ | Фильмы и сериалы,kino_mania,22798006
10,125402825,Artyom,Amosov,MDK,mudachyo,23148107
11,125402825,Artyom,Amosov,Artist Music,artistmusic,23162459
...,...,...,...,...,...,...
365358,794082984,Marina,Manerova,Путь к себе. Эзотерика.,prosnis24,46558304
365359,794082984,Marina,Manerova,Новости | Саранск City | Republic of Mordovia,gorod_saransk,36809318
365360,794082984,Marina,Manerova,САРАНСК Онлайн,rmsar,60178575
365361,794082984,Marina,Manerova,10 канал,10tvrm,70336234


In [41]:
import networkx as nx  # импортируем библиотеку
G = nx.Graph()         # создаем пустой граф

In [42]:
edgelist = df[['user_id', 'group_id']]

In [43]:
edgelist

,user_id,group_id
7,125402825,19603458
8,125402825,22778924
9,125402825,22798006
10,125402825,23148107
11,125402825,23162459
...,...,...
365358,794082984,46558304
365359,794082984,36809318
365360,794082984,60178575
365361,794082984,70336234


In [46]:
G1=nx.from_pandas_edgelist(edgelist,'user_id', 'group_id', create_using=nx.Graph())

In [56]:
centrality = sorted(list(nx.degree_centrality(G1).items()), key=lambda i: i[1], reverse=True)

In [73]:
centrality

[(91412129, 0.02702331091086009),
 (39325472, 0.020137919363709438),
 (28905875, 0.01622456318483733),
 (45745333, 0.011539112408512077),
 (49097575, 0.011020857130769555),
 (66678575, 0.0104285653847781),
 (73424524, 0.010259339171637686),
 (42122605, 0.009973769936963235),
 (192271135, 0.009688200702288784),
 (22798006, 0.009296865084401574),
 (39480832, 0.009000719211405847),
 (29573241, 0.008947836019799467),
 (147115743, 0.00888437618987181),
 (72081844, 0.008842069636586706),
 (89246551, 0.008577653678554807),
 (91050183, 0.008133434869061217),
 (73247559, 0.007995938570884629),
 (38801259, 0.007424800101535727),
 (57846937, 0.006980581292042137),
 (141959356, 0.006451749375978338),
 (27895931, 0.0062296399712315435),
 (12353330, 0.006060413758091128),
 (58170807, 0.00592291745991454),
 (64863616, 0.005721961331810297),
 (47110304, 0.005573888395312434),
 (214675919, 0.0055104285653847776),
 (24092182, 0.00547869865042095),
 (208984568, 0.0052671658839954305),
 (104597533, 0.0051

In [78]:
group_values_df = pd.DataFrame(centrality, columns=['group_id', 'weight'])

# Merge the two dataframes on 'group_id' to get the corresponding 'group_name'
merged_df = pd.merge(group_values_df, df, on='group_id', how='left')

# Create the list of tuples (group_id, weight, group_name)
result = [(row['group_id'], row['weight'], row['group_name']) for idx, row in merged_df.iterrows()]

result_df = pd.DataFrame(result, columns=['group_id', 'weight', 'group_name'])


In [79]:
result_df.drop_duplicates()

,group_id,weight,group_name
0,91412129,0.027023,Подслушано Рузаевка (САРАНСК)
2555,39325472,0.020138,Типичный Балаково!
4459,28905875,0.016225,Рифмы и Панчи
5993,45745333,0.011539,4ch
7084,49097575,0.011021,Молодёжный Балаково
...,...,...,...
340986,198285669,0.000011,Нейрографика. Психолог. Дарья Петровичева
340987,217306195,0.000011,Служба Семьи в Белорецком районе
340988,198725806,0.000011,"Живые открытки, видео."
340989,213430879,0.000011,Жильё для отдыха Дивноморск
